In [10]:
# Здравствуйте!
# Честно говоря, это мой первый опыт решения NLP-задачи классификации.
# Для решения задачи мной решено было использовать концепцию мешка слов и логистическую регрессию.
# Приступим!

In [11]:
# Подключаем библиотеки
%matplotlib inline
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score 
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer 
russian_stopwords = stopwords.words("russian")
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
# Заводим функцию для преобразования списка слов в строку
def list_to_str(arr):
    str_ = ''
    for i in arr:
        str_ += '{0} '.format(i)
    
    return str_

# Тут весь препроцессинг: перевод слов в нижний регистр, 
# удаление пунктуации и стоп-слов, стемматизация
def preprocess(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = text.lower()
    text = tokenizer.tokenize(text)
    stemmer = SnowballStemmer('russian')
    text = [word for word in text if not word in russian_stopwords]
    text = [stemmer.stem(word) for word in text]
    return list_to_str(text).strip()

In [13]:
train_data = pd.read_table('train.tsv')

In [14]:
train_data.head()

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1
1,Агент Кокорина назвал езду по встречке житейск...,0
2,Госдума рассмотрит возможность введения секрет...,1
3,ФАС заблокировала поставку скоростных трамваев...,0
4,Против Навального завели дело о недоносительст...,1


In [15]:
train_data['processed_title'] = train_data['title'].map(preprocess)

In [16]:
# Появляется третий столбец - предобработанные данные
train_data.head()

,title,is_fake,processed_title
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1,москвич владимир клутин пришел счет вмешательс...
1,Агент Кокорина назвал езду по встречке житейск...,0,агент кокорин назва езд встречк житейск истор
2,Госдума рассмотрит возможность введения секрет...,1,госдум рассмотр возможн введен секретн стат уг...
3,ФАС заблокировала поставку скоростных трамваев...,0,фас заблокирова поставк скоростн трамва москв
4,Против Навального завели дело о недоносительст...,1,прот навальн завел дел недоносительств волков


In [18]:
# Разбиваем данные на данные для обучения и для тестирования,
# Размер выборки для тестирования при этом составляет 15% от основной
X_train, X_test, y_train, y_test = train_test_split(train_data['processed_title'], train_data['is_fake'], test_size = .15)

In [19]:
# Формируем векторное представление, т.е. реализуем bag of words
vectorizer = CountVectorizer()

X_train_BOW = vectorizer.fit_transform(X_train)
X_test_BOW = vectorizer.transform(X_test)

In [20]:
# Обучаем логистическую регрессию с обычными параметрами
clf = LogisticRegression(random_state=0).fit(X_train_BOW, y_train)

In [21]:
y_predict_BOW = clf.predict(X_test_BOW)

In [22]:
# Проверяем результативность модели на метрике f1_score
f1_score(y_predict_BOW, y_test)

0.8287153652392947

In [51]:
# Результативность модели на f1_score получилась ~0.83, что меня устраивает,
# потому я перехожу к тестовым данным

In [59]:
test_data = pd.read_table('test.tsv')

In [60]:
test_data.head()

,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,0
1,Ночью под Минском на президентской горе Белара...,0
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,0
3,"Сельская церковь, собравшая рекордно низкое ко...",0
4,Акции Google рухнули после объявления о переза...,0


In [61]:
test_data['processed_title'] = test_data['title'].map(preprocess)

In [62]:
X_BOW = vectorizer.transform(test_data.processed_title)

In [63]:
y_BOW = clf.predict(X_BOW)

In [64]:
y_BOW

array([1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1,

In [65]:
test_data['is_fake'] = y_BOW

In [66]:
test_data = test_data.drop('processed_title', axis=1)

In [69]:
test_data.head()

,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,1
1,Ночью под Минском на президентской горе Белара...,1
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,1
3,"Сельская церковь, собравшая рекордно низкое ко...",0
4,Акции Google рухнули после объявления о переза...,0


In [68]:
test_data.to_csv('predictions.tsv', sep=',') 